In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np

In [3]:
# Parametry
batch_size = 64
learning_rate = 0.001
num_epochs = 20
l2_lambda = 0.01  # Współczynnik regularyzacji L2

# Transformacje danych
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Pobieranie danych
train_dataset = datasets.MNIST(root='data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
class SimpleFCNN(nn.Module):
    def __init__(self):
        super(SimpleFCNN, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = x.view(-1, 28*28)  # spłaszczenie obrazu
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Inicjalizacja modelu i kryterium
model = SimpleFCNN()
criterion = nn.CrossEntropyLoss()

In [7]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=l2_lambda)

In [9]:
def train(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0
        for images, labels in train_loader:
            # Zero gradientów
            optimizer.zero_grad()
            
            # Przekazanie przez sieć
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Obliczenie gradientów i aktualizacja wag
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        avg_loss = running_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

train(model, train_loader, criterion, optimizer, num_epochs)

Epoch [1/20], Loss: 0.4537
Epoch [2/20], Loss: 0.2918
Epoch [3/20], Loss: 0.2499
Epoch [4/20], Loss: 0.2323
Epoch [5/20], Loss: 0.2209
Epoch [6/20], Loss: 0.2146
Epoch [7/20], Loss: 0.2099
Epoch [8/20], Loss: 0.2049
Epoch [9/20], Loss: 0.2054
Epoch [10/20], Loss: 0.2030
Epoch [11/20], Loss: 0.1990
Epoch [12/20], Loss: 0.1987
Epoch [13/20], Loss: 0.1987
Epoch [14/20], Loss: 0.1966
Epoch [15/20], Loss: 0.1947
Epoch [16/20], Loss: 0.1940
Epoch [17/20], Loss: 0.1944
Epoch [18/20], Loss: 0.1927
Epoch [19/20], Loss: 0.1924
Epoch [20/20], Loss: 0.1943


In [11]:
def evaluate(model, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            
            # Przewidywanie
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
    
    test_loss /= len(test_loader)
    accuracy = correct / len(test_loader.dataset)
    print(f"Test Loss: {test_loss:.4f}, Accuracy: {accuracy * 100:.2f}%")

evaluate(model, test_loader, criterion)

Test Loss: 0.1724, Accuracy: 95.01%
